## Compounds produced by commmunity

In [1]:
import pandas as pd
import reframed
import os
import json
import sys

sys.path.append('../functions')

import translation_dicts
import general_functions as general_func

In [2]:
compounds_dict, source_dict,substrate_dict, gas_sheet_dict, community_dict = translation_dicts.translation_dicts()

In [3]:
compounds_dict['Isovalericacid(mg/L)'] = '3mb'
compounds_dict['Hydrogen'] = 'h2'
compounds_dict['CO2'] = 'co2'
compounds_dict

{'Aceticacid(mg/L)': 'ac',
 'Ethanol(mg/L)': 'etoh',
 'Propionicacid(mg/L)': 'ppa',
 'Butyricacid(mg/L)': 'but',
 'Lacticacid(mg/L)': 'lac__L',
 'Formicacid(mg/L)': 'for',
 'Valericacid(mg/L)': 'pta',
 '1-Propanol(mg/L)': 'ppoh',
 'Isovalericacid(mg/L)': '3mb',
 'Caproicacid(mg/L)': 'hxa',
 'Isobutyricacid(mg/L)': 'ibt',
 'Levulinicacid': '4oxptn',
 'Hydrogen': 'h2',
 'CO2': 'co2'}

In [4]:
compounds_dict_list = {name:[id_] for name,id_ in compounds_dict.items()}

compounds_dict_list["Isobutyricacid(mg/L)"] = ['isobuta','ibt','2mpa']
compounds_dict_list["Isovalericacid(mg/L)"] = ['3mb','3mb','ival']
compounds_dict_list['Caproicacid(mg/L)'] = ['caproic','hxa']
compounds_dict_list["Levulinicacid"] = ['4opa','4oxptn']

compounds_dict_list['Hydrogen'] = ['h2']
compounds_dict_list['CO2'] = ['co2']
compounds_dict_list

{'Aceticacid(mg/L)': ['ac'],
 'Ethanol(mg/L)': ['etoh'],
 'Propionicacid(mg/L)': ['ppa'],
 'Butyricacid(mg/L)': ['but'],
 'Lacticacid(mg/L)': ['lac__L'],
 'Formicacid(mg/L)': ['for'],
 'Valericacid(mg/L)': ['pta'],
 '1-Propanol(mg/L)': ['ppoh'],
 'Isovalericacid(mg/L)': ['3mb', '3mb', 'ival'],
 'Caproicacid(mg/L)': ['caproic', 'hxa'],
 'Isobutyricacid(mg/L)': ['isobuta', 'ibt', '2mpa'],
 'Levulinicacid': ['4opa', '4oxptn'],
 'Hydrogen': ['h2'],
 'CO2': ['co2']}

### Make overview of all compounds measured the enrichment cultures

The strategy for defining if the compound is produced or not is:
- Assuming that positive concertation==the compound is produced by the community

In [5]:
community_production_names = {}

# For each source 
for name, id_source in source_dict.items():
    
    # Read and process excel sheet
    ## Get the Excel sheet specifically for the source
    data_df = pd.read_excel("../input/files_from_fairdomhub/enrichment_cultures _data.xlsx",sheet_name=name)
    data_df.dropna(how='all', axis='columns',inplace=True)
    
    data_df = data_df.iloc[:,1:-1]
    data_df.set_index("Sample name ",inplace=True)
    data_df.columns = [col.replace(" ","") for col in data_df.columns]
    
    # For each substrate
    for name_sub,id_sub in substrate_dict.items():
        community_id = id_source+"_"+id_sub
                
        if community_id=="M_A": # Marshland on Avicel is not a combination in our experiments.
            continue
        
        community_production_names[community_id]={}
        data_df_sub_max = data_df[data_df.index.str.contains(community_id)].max()
        
        # For each compound in our analysis
        for compound in compounds_dict.keys():
            if compound not in data_df_sub_max.index:
                print(community_id)
                print(compound + " not in columns")
                continue
            
            # If max concentration is higher than 0 -> assume that it is produced by a community member
            if data_df_sub_max[compound]>0:
                community_production_names[community_id][compound]=1
                
        # These gasses where observed for all enrichment cultures. (for gasses CH4 is not included because it is not a part of the bacterial universal model)
        community_production_names[community_id]["Hydrogen"]=1
        community_production_names[community_id]["CO2"]=1

M_X
Hydrogen not in columns
M_X
CO2 not in columns
M_P
Hydrogen not in columns
M_P
CO2 not in columns
CD_X
Hydrogen not in columns
CD_X
CO2 not in columns
CD_A
Hydrogen not in columns
CD_A
CO2 not in columns
CD_P
Hydrogen not in columns
CD_P
CO2 not in columns
CM_X
Hydrogen not in columns
CM_X
CO2 not in columns
CM_A
Hydrogen not in columns
CM_A
CO2 not in columns
CM_P
Hydrogen not in columns
CM_P
CO2 not in columns


In [6]:
community_production_names

{'M_X': {'Aceticacid(mg/L)': 1,
  'Ethanol(mg/L)': 1,
  'Propionicacid(mg/L)': 1,
  'Butyricacid(mg/L)': 1,
  'Formicacid(mg/L)': 1,
  '1-Propanol(mg/L)': 1,
  'Isovalericacid(mg/L)': 1,
  'Caproicacid(mg/L)': 1,
  'Isobutyricacid(mg/L)': 1,
  'Hydrogen': 1,
  'CO2': 1},
 'M_P': {'Aceticacid(mg/L)': 1,
  'Ethanol(mg/L)': 1,
  'Butyricacid(mg/L)': 1,
  'Formicacid(mg/L)': 1,
  '1-Propanol(mg/L)': 1,
  'Caproicacid(mg/L)': 1,
  'Hydrogen': 1,
  'CO2': 1},
 'CD_X': {'Aceticacid(mg/L)': 1,
  'Ethanol(mg/L)': 1,
  'Propionicacid(mg/L)': 1,
  'Butyricacid(mg/L)': 1,
  'Lacticacid(mg/L)': 1,
  'Formicacid(mg/L)': 1,
  'Valericacid(mg/L)': 1,
  '1-Propanol(mg/L)': 1,
  'Caproicacid(mg/L)': 1,
  'Isobutyricacid(mg/L)': 1,
  'Levulinicacid': 1,
  'Hydrogen': 1,
  'CO2': 1},
 'CD_A': {'Aceticacid(mg/L)': 1,
  'Ethanol(mg/L)': 1,
  'Propionicacid(mg/L)': 1,
  'Butyricacid(mg/L)': 1,
  'Lacticacid(mg/L)': 1,
  '1-Propanol(mg/L)': 1,
  'Isovalericacid(mg/L)': 1,
  'Hydrogen': 1,
  'CO2': 1},
 'CD_P'

In [8]:
compounds_dict_list

{'Aceticacid(mg/L)': ['ac'],
 'Ethanol(mg/L)': ['etoh'],
 'Propionicacid(mg/L)': ['ppa'],
 'Butyricacid(mg/L)': ['but'],
 'Lacticacid(mg/L)': ['lac__L'],
 'Formicacid(mg/L)': ['for'],
 'Valericacid(mg/L)': ['pta'],
 '1-Propanol(mg/L)': ['ppoh'],
 'Isovalericacid(mg/L)': ['3mb', '3mb', 'ival'],
 'Caproicacid(mg/L)': ['caproic', 'hxa'],
 'Isobutyricacid(mg/L)': ['isobuta', 'ibt', '2mpa'],
 'Levulinicacid': ['4opa', '4oxptn'],
 'Hydrogen': ['h2'],
 'CO2': ['co2']}

### Save data 

In [15]:
with open("../output/community_production.json", "w") as outfile: 
    json.dump(community_production, outfile)

In [16]:
with open("../output/community_production_names.json", "w") as outfile: 
    json.dump(community_production_names, outfile)

In [17]:
with open("../output/compounds_dict_list.json", "w") as outfile: 
    json.dump(compounds_dict_list, outfile)

In [18]:
with open("../output/compounds_dict.json", "w") as outfile: 
    json.dump(compounds_dict, outfile)